In [1]:
from binn.NN import BINN

# Create the model
model = BINN(
            input_data  = 'data/TestQM.tsv', 
            pathways = 'data/pathways.tsv',
            translation_mapping  = 'data/translation.tsv',
            input_data_column = 'Protein',
            activation ='tanh', 
            learning_rate  = 1e-4, 
            n_layers  = 4, 
            scheduler = 'plateau',
            optimizer = 'adam',
            validate  = True,
            n_outputs = 2)

/home/erikh/BINN/BINN/test-venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of reactome ids before subsetting: 3484
Unique proteins in reactome df: 458
Function called 1 times.
Values in idx_list: 0
Function called 2 times.
Values in idx_list: 652
Function called 3 times.
Values in idx_list: 989
Function called 4 times.
Values in idx_list: 1173
Function called 5 times.
Values in idx_list: 1264
Function called 6 times.
Values in idx_list: 1304
Function called 7 times.
Values in idx_list: 1322
Function called 8 times.
Values in idx_list: 1328
Function called 9 times.
Values in idx_list: 1329
Base case reached
Final number of unique connections in pathway:  1003
Network:  DiGraph with 1015 nodes and 1031 edges
Number of copies made for 4 layers: 26


In [6]:
# Train the model
import pandas as pd
from UtilForExamples import fit_protein_matrix_to_network_input, generate_data
import torch
from pytorch_lightning import Trainer

design_matrix = pd.read_csv('data/design_matrix.tsv', sep="\t")
protein_matrix = pd.read_csv('data/TestQM.tsv', sep='\t')

protein_matrix = fit_protein_matrix_to_network_input(
    protein_matrix, features=model.RN.inputs)

X, y = generate_data(protein_matrix, design_matrix=design_matrix)
dataloader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(torch.Tensor(X), torch.LongTensor(y)),
                                            batch_size=8,
                                            num_workers=12,
                                            shuffle=True)
trainer = Trainer(max_epochs=20)
trainer.fit(model, dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/erikh/BINN/BINN/test-venv/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/home/erikh/BINN/BINN/test-venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:107: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
Missing logger folder: /home/erikh/BINN/BINN/lightning_logs

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 364 K 
1 | loss   | CrossEntropyLoss | 0     
--------------------------------------------
364 K     Trainable params
0         Non-trainable params
364 K     Total params
1.457     T

Epoch 0:   0%|          | 0/25 [00:00<?, ?it/s] 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x6647 and 449x443)